# Requerimiento 2: Análisis de Similitud entre Artículos

## Descripción del Requerimiento

Este notebook implementa un **análisis comparativo de algoritmos de similitud textual** para evaluar la similitud entre abstracts de artículos científicos.

### Objetivos:
1. Cargar y preprocesar archivo BibTeX consolidado
2. Eliminar duplicados por título
3. Comparar 6 algoritmos de similitud textual
4. Interpretar resultados con umbrales de similitud

### Categorías de Algoritmos:

#### Algoritmos Clásicos (No supervisados):
1. **Levenshtein**: Distancia de edición entre caracteres
2. **Coseno TF-IDF**: Similitud vectorial con pesos TF-IDF
3. **Jaccard**: Intersección de conjuntos de palabras
4. **Euclidiana**: Distancia en espacio vectorial

#### Algoritmos Basados en IA (Supervisados):
5. **SBERT**: Embeddings semánticos con Sentence-BERT
6. **Cross-Encoder**: Evaluación directa de pares de texto

### Umbrales de Interpretación:

| Rango | Interpretación |
|-------|----------------|
| ≥ 0.80 | Muy similares |
| 0.50 - 0.79 | Similitud moderada |
| 0.20 - 0.49 | Poco similares |
| < 0.20 | No similares |

### Carga y Deduplicación del Archivo BibTeX

Esta celda carga el archivo consolidado y elimina artículos duplicados.

#### Proceso:

##### 1. Configuración del Parser
```python
parser = bibtexparser.bparser.BibTexParser(common_strings=True)
parser.expect_multiple_parse = True
```
- **`common_strings=True`**: Expande abreviaturas comunes de BibTeX
- **`expect_multiple_parse=True`**: Suprime warnings de múltiples entradas

##### 2. Lectura del Archivo
```python
with open(CONSOLIDADO_PATH, encoding="utf-8") as f:
    bib_database = bibtexparser.load(f, parser=parser)
```
- **Encoding UTF-8**: Soporta caracteres especiales
- **Context manager**: Cierre automático del archivo

##### 3. Extracción de Campos
```python
for entry in bib_database.entries:
    data.append({
        "title": entry.get("title", "").strip(),
        "authors": entry.get("author", ""),
        "keywords": entry.get("keywords", ""),
        "abstract": entry.get("abstract", "")
    })
```
- Extrae título, autores, keywords y abstract
- Manejo de valores faltantes con `entry.get(campo, "")`

##### 4. Deduplicación por Título
```python
df_unique = df.drop_duplicates(subset="title", keep="first")
```
- **`keep="first"`**: Mantiene la primera aparición de cada título
- Los títulos son únicos en publicaciones académicas

##### 5. Guardado de Resultados
```python
df_unique.to_csv("articulos_unicos.csv", index=False)
df_duplicates.to_csv("articulos_repetidos.csv", index=False)
```
- **`articulos_unicos.csv`**: Dataset limpio para análisis
- **`articulos_repetidos.csv`**: Duplicados para auditoría

In [1]:
import bibtexparser
import pandas as pd
import os
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Obtener ruta del archivo consolidado
CONSOLIDADO_PATH = os.getenv("CONSOLIDADO_PATH", "../salidas/consolidado.bib")

# Verificar que el archivo existe
if not os.path.exists(CONSOLIDADO_PATH):
    print(f"Error: No se encuentra el archivo en {CONSOLIDADO_PATH}")
    print(f"Directorio actual: {os.getcwd()}")
    print(f"Archivos disponibles:")
    # Buscar archivos .bib en directorios comunes
    for root, dirs, files in os.walk(".."):
        for file in files:
            if file.endswith("consolidado.bib"):
                print(f"Encontrado: {os.path.join(root, file)}")
    raise FileNotFoundError(f"No se encuentra {CONSOLIDADO_PATH}")

print(f"Leyendo archivo: {CONSOLIDADO_PATH}")

# Configurar el parser
parser = bibtexparser.bparser.BibTexParser(common_strings=True)
parser.expect_multiple_parse = True  # Evita el warning

# Leer TODO el archivo de una vez
with open(CONSOLIDADO_PATH, encoding="utf-8") as f:
    bib_database = bibtexparser.load(f, parser=parser)

# Extraer la información
data = []
for entry in bib_database.entries:
    data.append({
        "title": entry.get("title", "").strip(),
        "authors": entry.get("author", ""),
        "keywords": entry.get("keywords", ""),
        "abstract": entry.get("abstract", "")
    })

df = pd.DataFrame(data)

# Eliminar duplicados por título
df_unique = df.drop_duplicates(subset="title", keep="first")
df_duplicates = df[df.duplicated(subset="title", keep=False)]

# Guardar resultados
df_unique.to_csv("articulos_unicos.csv", index=False)
df_duplicates.to_csv("articulos_repetidos.csv", index=False)

print(f"Artículos totales: {len(df)}")
print(f"Artículos únicos: {len(df_unique)}")
print(f"Artículos repetidos: {len(df_duplicates)}")

df_unique.head()

Leyendo archivo: /home/yep/Documentos/proyectoAnalisisAlgoritmos/proyecto/consolidado.bib
Artículos totales: 10226
Artículos únicos: 10189
Artículos repetidos: 38


,title,authors,keywords,abstract
0,Do Robots Dream of Passing a Programming Course?,"Torres, Nicolás",Training;Computational modeling;Instruments;Na...,Programming typically involves humans formulat...
1,WeAIR: Wearable Swarm Sensors for Air Quality ...,"Dimitri, Giovanna Maria and Parri, Lorenzo and...",Temperature measurement;Climate change;Cloud c...,The present study proposes the implementation ...
2,Discriminative-Generative Representation Learn...,"Li, Duanjiao and Chen, Yun and Zhang, Ying and...",Representation learning;Semantics;Asia;Self-su...,"Generative Adversarial Networks (GANs), as a f..."
3,3 Generative AI Models and LLM: Training Techn...,"Arun, C. and Karthick, S. and Selvakumara Samy...",,Generative artificial intelligence (AI) has be...
4,Virtual Human: A Comprehensive Survey on Acade...,"Cui, Lipeng and Liu, Jiarui",Digital humans;Motion capture;Face recognition...,As a creative method for virtual human individ...


### Selección de Artículos para Comparación

Esta celda selecciona una muestra de artículos para análisis de similitud.

#### Estrategia de Selección:

```python
abstracts = df["abstract"].head(3).tolist()
titles = df["title"].head(3).tolist()
```

**Características**:
- Toma los primeros N artículos del DataFrame
- Simple y reproducible
- Puede ajustarse según necesidad (`.head(N)` o `.sample(N)`)

#### Alternativas de Selección:

**Selección Aleatoria**:
```python
indices = random.sample(range(len(df)), 3)
abstracts = df.iloc[indices]["abstract"].tolist()
```

**Selección por Keywords**:
```python
mask = df["keywords"].str.contains("generative", case=False, na=False)
sample = df[mask].head(3)
```

**Selección por Longitud**:
```python
df["abstract_len"] = df["abstract"].str.len()
sample = df[(df["abstract_len"] > 500) & (df["abstract_len"] < 700)].head(3)
```

In [2]:
# Ejemplo: selecionando los primeros 3
abstracts = df["abstract"].head(3).tolist()
titles = df["title"].head(3).tolist()

for i, t in enumerate(titles):
    print(f"{i}. {t}\n{abstracts[i][:200]}...\n")


0. Do Robots Dream of Passing a Programming Course?
Programming typically involves humans formulating instructions for a computer to execute computations. If we adhere to this definition, a machine would seemingly lack the capability to autonomously de...

1. WeAIR: Wearable Swarm Sensors for Air Quality Monitoring to Foster Citizens' Awareness of Climate Change
The present study proposes the implementation of an air quality measurement tool through the use of a swarm of wearable devices, named WeAIR, consisting of wearable sensors for measuring NOx, CO2, CO,...

2. Discriminative-Generative Representation Learning for One-Class Anomaly Detection
Generative Adversarial Networks (GANs), as a form of generative self-supervised learning, have garnered significant attention in anomaly detection. However, the generator's capacity for representation...



### Algoritmo 1: Distancia de Levenshtein

La **distancia de Levenshtein** mide el número mínimo de operaciones de edición (inserción, eliminación, sustitución) necesarias para transformar un texto en otro.

#### Fundamento Matemático

Para dos cadenas **a** y **b**, la distancia de Levenshtein se define recursivamente:

```
lev(a,b) = {
    |a|                                  si |b| = 0
    |b|                                  si |a| = 0
    lev(tail(a), tail(b))                si a[0] = b[0]
    1 + min {
        lev(tail(a), b)                  (eliminación)
        lev(a, tail(b))                  (inserción)
        lev(tail(a), tail(b))            (sustitución)
    }                                    en otro caso
}
```

#### Conversión a Similitud

```python
similarity = 1 - (distance / max_length)
```

**Rango**: [0, 1]
- **1.0** = textos idénticos
- **0.0** = textos completamente diferentes

#### Implementación

```python
import Levenshtein

def levenshtein_similarity(text1, text2):
    dist = Levenshtein.distance(text1, text2)
    max_len = max(len(text1), len(text2))
    similarity = 1 - dist / max_len
    return similarity
```

**Complejidad**: O(m × n) donde m, n son longitudes de los textos

#### Ventajas
- Simplicidad y facilidad de implementación
- Determinístico (siempre da el mismo resultado)
- Sensible a errores ortográficos
- No requiere entrenamiento

#### Desventajas
- Opera a nivel de caracteres, no entiende semántica
- Sensible a longitud de textos
- No detecta sinónimos o paráfrasis
- El orden de palabras afecta significativamente

In [3]:
import Levenshtein

def levenshtein_similarity(text1, text2):
    dist = Levenshtein.distance(text1, text2)
    max_len = max(len(text1), len(text2))
    similarity = 1 - dist / max_len
    
    # Interpretación del resultado
    if similarity >= 0.8:
        interpretation = "Los textos son muy similares."
    elif similarity >= 0.5:
        interpretation = "Los textos tienen cierta similitud moderada."
    elif similarity >= 0.2:
        interpretation = "Los textos son poco similares."
    else:
        interpretation = "Los textos no son similares."
    
    print(f"Similitud de Levenshtein: {similarity:.3f}")
    print(f"Interpretación: {interpretation}")
    return similarity


levenshtein_similarity(abstracts[0], abstracts[1])


Similitud de Levenshtein: 0.219
Interpretación: Los textos son poco similares.


0.21857923497267762

### Algoritmo 2: Similitud del Coseno con TF-IDF

La **similitud del coseno** mide el ángulo entre dos vectores en un espacio multidimensional. Combinada con **TF-IDF**, captura la importancia relativa de las palabras.

#### Fundamento Matemático

**TF-IDF (Term Frequency - Inverse Document Frequency)**

```
TF(t, d) = (Frecuencia de término t en documento d) / (Total de términos en d)
IDF(t, D) = log(Total de documentos / Documentos que contienen t)
TF-IDF(t, d, D) = TF(t, d) × IDF(t, D)
```

**Similitud del Coseno**

Para dos vectores **A** y **B**:

```
cos(θ) = (A · B) / (||A|| × ||B||)
```

Donde:
- **A · B** = producto punto = Σ(Aᵢ × Bᵢ)
- **||A||** = norma = √(Σ Aᵢ²)
- **θ** = ángulo entre vectores

**Rango**: [-1, 1]
- **1**: Vectores idénticos (θ = 0°)
- **0**: Vectores ortogonales (θ = 90°)
- **-1**: Vectores opuestos (θ = 180°)

#### Implementación

```python
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(abstracts)
cosine_sim = cosine_similarity(tfidf_matrix)
```

**Parámetros importantes**:
- **`stop_words='english'`**: Elimina palabras comunes (the, and, is...)
- **`max_features`**: Limita vocabulario
- **`ngram_range`**: Incluye n-gramas

#### Interpretación de Resultados

**Matriz de Similitud**:
- **Diagonal**: Siempre 1.0 (cada documento consigo mismo)
- **Valores fuera de diagonal**: Similitud entre pares de documentos

#### Ventajas
- Opera a nivel de palabras, no caracteres
- TF-IDF pondera palabras importantes
- Normalizado por magnitud de vectores
- Eficiente y escalable
- Interpretable

#### Desventajas
- No captura similitud semántica (sinónimos)
- Bag of words: ignora orden de palabras
- Vocabulario cerrado (solo palabras vistas)
- Vectores dispersos (sparse)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def interpretar_similitud(valor):
    """Interpreta el nivel de similitud según el valor de coseno."""
    if valor >= 0.8:
        return "Los textos son muy similares."
    elif valor >= 0.5:
        return "Los textos tienen similitud moderada."
    elif valor >= 0.2:
        return "Los textos son poco similares."
    else:
        return "Los textos no son similares."

# --- Cálculo del TF-IDF y similitud ---
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(abstracts)
cosine_sim = cosine_similarity(tfidf_matrix)

# --- Impresión de resultados ---
print("Matriz de similitud de coseno:")
print(cosine_sim)
print("\nInterpretación par a par:\n")

# Recorre cada par de textos (sin repetir)
for i in range(len(abstracts)):
    for j in range(i + 1, len(abstracts)):
        valor = cosine_sim[i][j]
        print(f"Similitud entre Abstract {i} y Abstract {j}: {valor:.3f} → {interpretar_similitud(valor)}")


Matriz de similitud de coseno:
[[1.         0.0429688  0.09021994]
 [0.0429688  1.         0.00515135]
 [0.09021994 0.00515135 1.        ]]

Interpretación par a par:

Similitud entre Abstract 0 y Abstract 1: 0.043 → Los textos no son similares.
Similitud entre Abstract 0 y Abstract 2: 0.090 → Los textos no son similares.
Similitud entre Abstract 1 y Abstract 2: 0.005 → Los textos no son similares.


### Algoritmo 3: Similitud de Jaccard

El **índice de Jaccard** mide la similitud entre conjuntos calculando la proporción de elementos comunes.

#### Fundamento Matemático

Para dos conjuntos **A** y **B**:

```
J(A, B) = |A ∩ B| / |A ∪ B|
```

Donde:
- **A ∩ B** = Intersección (elementos en ambos)
- **A ∪ B** = Unión (elementos en al menos uno)
- **| |** = Cardinalidad (número de elementos)

**Rango**: [0, 1]
- **1**: Conjuntos idénticos
- **0**: Sin elementos comunes

#### Ejemplo

**Texto 1**: "machine learning models"
**Texto 2**: "deep learning networks"

```
A = {machine, learning, models}
B = {deep, learning, networks}
A ∩ B = {learning}                           → |A ∩ B| = 1
A ∪ B = {machine, learning, models, deep, networks} → |A ∪ B| = 5
J(A, B) = 1 / 5 = 0.20
```

#### Implementación

```python
def jaccard_similarity(a, b):
    a_set = set(a.lower().split())
    b_set = set(b.lower().split())
    intersection = len(a_set & b_set)
    union = len(a_set | b_set)
    return intersection / union
```

**Operaciones de conjuntos en Python**:
- `&` = Intersección
- `|` = Unión
- `set()` = Elimina duplicados automáticamente

#### Ventajas
- Muy simple de entender e implementar
- Rápido: operaciones de conjuntos son O(n)
- Simétrico: J(A,B) = J(B,A)
- No requiere pesos ni entrenamiento
- Robusto a duplicados

#### Desventajas
- Ignora frecuencia de palabras
- Ignora orden de palabras
- No captura similitud semántica
- Sensible a longitud de textos
- Cuenta stopwords igual que palabras importantes

In [5]:
def jaccard_similarity(a, b):
    # Convertir a conjuntos de palabras
    a_set, b_set = set(a.lower().split()), set(b.lower().split())
    similarity = len(a_set & b_set) / len(a_set | b_set)
    
    # Interpretación de la similitud
    if similarity >= 0.8:
        interpretation = "Los textos son muy similares."
    elif similarity >= 0.5:
        interpretation = "Los textos tienen similitud moderada."
    elif similarity >= 0.2:
        interpretation = "Los textos son poco similares."
    else:
        interpretation = "Los textos no son similares."
    
    # Mostrar resultados
    print(f"Similitud de Jaccard: {similarity:.3f}")
    print(f"Interpretación: {interpretation}")
    return similarity

# Ejemplo de uso
jaccard_similarity(abstracts[0], abstracts[1])


Similitud de Jaccard: 0.078
Interpretación: Los textos no son similares.


0.0782122905027933

### Algoritmo 4: Similitud Euclidiana

La **distancia euclidiana** mide la distancia geométrica entre dos puntos en un espacio multidimensional.

#### Fundamento Matemático

**Distancia Euclidiana**:

Para dos vectores **A** y **B** de dimensión n:

```
d(A, B) = √(Σᵢ (Aᵢ - Bᵢ)²)
```

**Conversión a Similitud**:

```
similarity = 1 / (1 + distance)
```

**Rango**: [0, 1]
- **1**: Vectores idénticos (distancia = 0)
- **→0**: Vectores muy diferentes (distancia → ∞)

#### Implementación

```python
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances

vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(abstracts)
distances = euclidean_distances(X)
similarities = 1 / (1 + distances)
```

**Diferencia con TF-IDF**:
- **CountVectorizer**: Frecuencias simples (1, 2, 3...)
- **TfidfVectorizer**: Frecuencias ponderadas

#### Ventajas
- Intuitivo: distancia geométrica familiar
- Sensible a magnitud (captura diferencias de frecuencia)
- Fácil de visualizar en 2D/3D

#### Desventajas
- Sensible a dimensionalidad alta
- Sensible a escala (palabras frecuentes dominan)
- No normalizado por longitud de texto
- Menos usado que coseno para textos

## Algoritmo de Similitud Euclidiana

### Algoritmo 5: SBERT (Sentence-BERT)

**SBERT** es un modelo de IA que convierte textos en **embeddings semánticos** de alta calidad, capturando el significado más allá de las palabras exactas.

#### Fundamento Conceptual

**Embeddings** son representaciones vectoriales densas que capturan significado semántico:

```
"car" → [0.23, -0.45, 0.67, ..., 0.12]  (384 dimensiones)
"automobile" → [0.25, -0.43, 0.69, ..., 0.14]  (similar)
```

**Propiedad clave**: Vectores de palabras similares están cerca en el espacio

#### Arquitectura SBERT

Basado en BERT (Bidirectional Encoder Representations from Transformers):

```
Texto → Tokenización → BERT → Mean Pooling → Embedding (384D)
```

**Componentes**:
1. **Tokenización**: Divide texto en subpalabras (WordPiece)
2. **BERT**: Transformer que procesa contexto bidireccional
3. **Mean Pooling**: Promedia embeddings de tokens
4. **Normalización**: Vector unitario (norma L2 = 1)

#### Cálculo de Similitud

Similitud del Coseno entre Embeddings:

```
similarity = cos(θ) = (E₁ · E₂) / (||E₁|| × ||E₂||)
```

**Ventaja sobre TF-IDF**: Captura similitud semántica, no solo léxica

#### Implementación

```python
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts, convert_to_tensor=True)
similarity = util.cos_sim(embeddings[0], embeddings[1]).item()
```

**Modelo**: `all-MiniLM-L6-v2`
- **Tamaño**: 22.7 MB
- **Dimensiones**: 384
- **Velocidad**: ~3,000 oraciones/segundo (GPU)

#### Ventajas
- Comprende semántica: entiende sinónimos y paráfrasis
- Considera contexto de las palabras
- Modelos multilingües disponibles
- Pre-entrenado, no requiere entrenamiento adicional
- Embeddings se calculan una vez y se reusan

#### Desventajas
- Computacionalmente costoso (requiere GPU para datasets grandes)
- Modelo ocupa ~100-500 MB en RAM
- Difícil interpretar por qué dos textos son similares
- Requiere dependencias (PyTorch, transformers)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances

# Vectorizar los textos (sin stopwords en inglés)
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(abstracts)

# Calcular distancias euclidianas
distances = euclidean_distances(X)

# Convertir a similitud (1 / (1 + distancia))
similarities = 1 / (1 + distances)

# Mostrar matriz de similitudes con interpretación
print("Matriz de similitud euclidiana:")
print(similarities)

# Interpretar los valores (solo pares distintos)
for i in range(len(abstracts)):
    for j in range(i + 1, len(abstracts)):
        sim = similarities[i, j]
        if sim >= 0.8:
            interpretation = "Los textos son muy similares."
        elif sim >= 0.5:
            interpretation = "Los textos tienen similitud moderada."
        elif sim >= 0.2:
            interpretation = "Los textos son poco similares."
        else:
            interpretation = "Los textos no son similares."
        
        print(f"\nSimilitud entre Abstract {i} y {j}: {sim:.3f}")
        print(f"Interpretación: {interpretation}")


Matriz de similitud euclidiana:
[[1.         0.06972692 0.06005113]
 [0.06972692 1.         0.05432747]
 [0.06005113 0.05432747 1.        ]]

Similitud entre Abstract 0 y 1: 0.070
Interpretación: Los textos no son similares.

Similitud entre Abstract 0 y 2: 0.060
Interpretación: Los textos no son similares.

Similitud entre Abstract 1 y 2: 0.054
Interpretación: Los textos no son similares.


# algoritmos de similitud basados en IA

### Algoritmo 6: Cross-Encoder

**Cross-Encoder** es un modelo de IA que evalúa **directamente** la relación entre dos textos, sin generar embeddings intermedios.

#### Diferencia con SBERT

**SBERT (Bi-Encoder)**:
```
Texto A → Embedding A ─┐
                        ├─→ Similitud del Coseno
Texto B → Embedding B ─┘
```
- Embeddings se calculan una vez, se reusan
- Similitud indirecta (coseno de embeddings)

**Cross-Encoder**:
```
[Texto A ; Texto B] → Transformer → Score directo
```
- Evaluación directa, más precisa
- Debe procesar cada par individualmente

#### Arquitectura

**Proceso**:

1. **Concatenación**: `[CLS] Texto A [SEP] Texto B [SEP]`
2. **Tokenización**: Convierte a IDs de tokens
3. **BERT**: Procesa secuencia completa
4. **Clasificación**: Capa final predice score
5. **Normalización**: Convierte a probabilidad [0, 1]

**Fórmula**: `score = f([A ; B])` donde `f` es el transformer que procesa ambos textos conjuntamente

#### Implementación

```python
from sentence_transformers import CrossEncoder
import numpy as np

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
pair = [[text1, text2]]
score = cross_encoder.predict(pair)[0]
prob = 1 / (1 + np.exp(-score))  # Normalización con sigmoid
```

**Modelo**: `ms-marco-MiniLM-L-6-v2`
- **Entrenamiento**: MS MARCO dataset (500K+ pares)
- **Tamaño**: 90 MB
- **Velocidad**: ~100 pares/segundo (CPU)

#### Normalización del Score

Cross-Encoder retorna valores en rango (-∞, +∞). Se normaliza con función sigmoide:

```
σ(x) = 1 / (1 + e^(-x))
```

#### Ventajas
- Máxima precisión en benchmarks
- Atención cruzada: modela interacciones entre textos
- Score directo de relevancia
- Estado del arte en tareas de ranking

#### Desventajas
- No escalable: debe procesar cada par individualmente
- Sin embeddings reutilizables
- 10-100x más lento que SBERT
- Límite de tokens: máximo ~512 tokens (A + B combinados)

#### Estrategia Híbrida Recomendada

Pipeline de 2 etapas:

```python
# Etapa 1: Filtrado rápido con SBERT
embeddings = sbert_model.encode(all_documents)
top_100 = similarities.topk(100)

# Etapa 2: Re-ranking preciso con Cross-Encoder
pairs = [[query, doc] for doc in top_100_documents]
scores = cross_encoder.predict(pairs)
```

**Beneficios**: Velocidad de SBERT + Precisión de Cross-Encoder

### SBERT (Sentence-BERT) - Similitud Semántica con Embeddings

SBERT convierte cada texto en un vector numérico (embedding) en un espacio semántico. Luego compara esos vectores usando la similitud del coseno.

Valores cercanos a 1 indican textos muy similares.

**Referencia**: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

### Análisis Comparativo de Algoritmos

Este análisis compara el rendimiento de 6 algoritmos de similitud textual.

#### Categorías de Algoritmos

**Algoritmos Clásicos (No supervisados)**:
- **Levenshtein**: Distancia de edición a nivel de caracteres
- **TF-IDF Coseno**: Similitud vectorial con ponderación de términos
- **Jaccard**: Intersección de conjuntos de palabras
- **Euclidiana**: Distancia geométrica en espacio vectorial

**Algoritmos Basados en IA (Supervisados)**:
- **SBERT**: Embeddings semánticos con Sentence-BERT
- **Cross-Encoder**: Evaluación directa de pares de texto

#### Comparación de Características

| Algoritmo | Velocidad | Precisión | Semántica | Escalabilidad |
|-----------|-----------|-----------|-----------|---------------|
| **Levenshtein** | Muy alta | Baja | No | Alta |
| **TF-IDF Coseno** | Alta | Media | No | Alta |
| **Jaccard** | Muy alta | Baja | No | Alta |
| **Euclidiana** | Alta | Baja | No | Alta |
| **SBERT** | Media | Alta | Sí | Media |
| **Cross-Encoder** | Baja | Muy alta | Sí | Baja |

#### Guía de Selección

**Detección de duplicados exactos**: Levenshtein o Jaccard
- Rápidos, detectan copias casi exactas

**Búsqueda en base de datos (10K+ documentos)**: TF-IDF Coseno
- Balance velocidad/precisión, escalable

**Recomendación de artículos similares**: SBERT
- Captura similitud semántica, embeddings reutilizables

**Ranking de relevancia (Top-K)**: SBERT + Cross-Encoder
- SBERT para filtrado, Cross-Encoder para re-ranking

**Análisis exploratorio rápido**: Jaccard o TF-IDF
- Implementación simple, resultados inmediatos

#### Pipeline Híbrido Recomendado

```python
# Paso 1: Filtrado rápido con TF-IDF
tfidf_sim = cosine_similarity(tfidf_matrix)
candidates = tfidf_sim > 0.3

# Paso 2: Validación semántica con SBERT
sbert_embeddings = model.encode(candidate_pairs)
sbert_sim = util.cos_sim(sbert_embeddings)
final_similar = sbert_sim > 0.7

# Paso 3 (Opcional): Re-ranking con Cross-Encoder
cross_scores = cross_encoder.predict(final_pairs)
```

**Justificación**:
- TF-IDF elimina pares obviamente diferentes (rápido)
- SBERT valida similitud semántica (preciso)
- Cross-Encoder proporciona ranking final de alta calidad (opcional)

In [7]:

from sentence_transformers import SentenceTransformer, util
import numpy as np

# --- Textos (abstracts) ---
texts = [abstracts[0], abstracts[1]]  # puedes cambiar a tus abstracts

# --- Cargar modelo SBERT ---
model = SentenceTransformer('all-MiniLM-L6-v2')

# --- Obtener embeddings ---
embeddings = model.encode(texts, convert_to_tensor=True)

# --- Calcular similitud de coseno ---
similarity = util.cos_sim(embeddings[0], embeddings[1]).item()

# --- Interpretar el resultado ---
if similarity >= 0.8:
    interpretation = "Los textos son muy similares."
elif similarity >= 0.5:
    interpretation = "Los textos tienen similitud moderada."
elif similarity >= 0.2:
    interpretation = "Los textos son poco similares."
else:
    interpretation = "Los textos no son similares."

print(f"Similitud SBERT: {similarity:.3f}")
print(f"Interpretación: {interpretation}")


Similitud SBERT: 0.189
Interpretación: Los textos no son similares.


### Cross-Encoder - Evaluación Directa de Pares de Texto

El modelo recibe ambos textos juntos y predice una puntuación directa de similitud. Se entrena para entender la relación entre oraciones, no solo las palabras.

**Fórmula**: f([A;B]) es la representación conjunta del par dentro del transformer.

**Referencia**: https://huggingface.co/cross-encoder/ms-marco-MiniLM-L6-v2

In [8]:
from sentence_transformers import CrossEncoder
import numpy as np

# --- Cargar modelo ---
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# --- Evaluar similitud directa entre dos textos ---
pair = [[abstracts[0], abstracts[1]]]
score = cross_encoder.predict(pair)[0]

# --- Normalizar (si el score no está entre 0 y 1) ---
prob = 1 / (1 + np.exp(-score)) if score > 1 or score < 0 else score

# --- Interpretación ---
if prob >= 0.8:
    interpretation = "Los textos son muy similares."
elif prob >= 0.5:
    interpretation = "Los textos tienen similitud moderada."
elif prob >= 0.2:
    interpretation = "Los textos son poco similares."
else:
    interpretation = "Los textos no son similares."

print(f"Puntaje Cross-Encoder: {score:.3f}")
print(f"Probabilidad normalizada: {prob:.3f}")
print(f"Interpretación: {interpretation}")


Puntaje Cross-Encoder: -6.766
Probabilidad normalizada: 0.001
Interpretación: Los textos no son similares.
